### Creating gpkg of power structures in OpenStreetMap

Saving power structures in the lines and points layers as buffered polygons, ready for masking in TOW Wales.

In [7]:
import geopandas as gpd
import pandas as pd
from shapely.ops import polygonize

In [5]:
##############################################
# Line features
##############################################

osm_fp = f'C:/Users/eleanor.downer/OneDrive - Forest Research/Documents/TOW_Wales/wales_osm_clean.gpkg'

osm_lines = gpd.read_file(osm_fp, layer='lines') # Open lines layer
osm_lines = osm_lines.to_crs(epsg=27700) # Reproject to EPSG:27700 (British National Grid)

osm_lines['other_tags'] = osm_lines['other_tags'].astype(str) # Convert 'other_tags' values to string
osm_power_features = osm_lines[osm_lines['other_tags'].str.contains("power", na=False)].copy() # Features that contain 'power' in 'other_tags'

osm_power_features['power'] = "" # Create new column
osm_power_features.drop(
    columns=['highway', 'waterway', 'aerialway', 'barrier', 'man_made', 'z_order'],
    inplace=True) # Drop other columns

for index, row in osm_power_features.iterrows(): # Itterate over rows

    # Find the power tag in other_tags and the value
    tags = row['other_tags']
    split_tags = tags.split(',')
    for tags in split_tags:
        if '"power"=>' in tags:
            power_type = tags.split('"')[-2]
    osm_power_features.at[index, 'power'] = power_type # Save power tag as attribute in 'power' column

structure_list = ['compensator', 'generator', 'plant', 'substation', 'switchgear'] # Power structures to be masked out
osm_power_structures = osm_power_features[osm_power_features['power'].isin(structure_list)].copy()

osm_power_cables = osm_power_features[osm_power_features['power'].isin(['cable', 'line', 'minor_line', 'minor_underground_cable'])].copy()

# Convert MultiLineStrings to Polygons - Function
def multiline_to_polygon(mls):
    polys = list(polygonize(mls))
    if len(polys) == 1:
        return polys[0]  # single polygon
    elif len(polys) > 1:
        # if multiple polygons are formed, return as MultiPolygon
        from shapely.geometry import MultiPolygon
        return MultiPolygon(polys)
    else:
        return None  # couldn't polygonize

# Convert to lines to polygons
osm_power_polygons = osm_power_structures.copy()
osm_power_polygons["geometry"] = osm_power_polygons["geometry"].apply(multiline_to_polygon)
osm_power_polygons['geometry'] = osm_power_polygons.buffer(2) # Buffer by 2m


In [11]:
###########################################
# Point features
###########################################

osm_points = gpd.read_file(osm_fp, layer='points') # Open points layer
osm_points = osm_points.to_crs(epsg=27700) # Reproject to EPSG:27700 (British National Grid)

osm_points['other_tags'] = osm_points['other_tags'].astype(str)
osm_power_points = osm_points[osm_points['other_tags'].str.contains("power", na=False)].copy()

for index, row in osm_power_points.iterrows():

    tags = row['other_tags']
    split_tags = tags.split(',')
    for tags in split_tags:
        if '"power"=>' in tags:
            power_type = tags.split('"')[-2]

    osm_power_points.at[index, 'power'] = power_type

osm_towers_and_poles = osm_power_points[osm_power_points['power'].isin(['pole', 'tower'])].copy()
osm_towers_and_poles = osm_towers_and_poles[osm_towers_and_poles['highway'].isna()]

poles = osm_towers_and_poles[osm_towers_and_poles['power'] == 'pole'].copy()
towers = osm_towers_and_poles[osm_towers_and_poles['power'] == 'tower'].copy()

poles['geometry'] = poles.buffer(2.5) # 5m buffer for poles
towers['geometry'] = towers.buffer(5) # 10m buffer for towers

osm_towers_and_poles = pd.concat([poles, towers], ignore_index=True)

# Make sure geometry type is Polygon/Multipolygon
osm_towers_and_poles = osm_towers_and_poles.set_geometry('geometry')
osm_towers_and_poles = osm_towers_and_poles.explode(index_parts=False)

osm_towers_and_poles.reset_index(drop=True, inplace=True) # reset index for clean output

osm_towers_and_poles.drop(
    columns=['barrier', 'highway', 'ref', 'address', 'is_in', 'place', 'man_made'],
    inplace=True) # Drop other columns


In [12]:
print(osm_power_polygons.columns)
print(osm_towers_and_poles.columns)

Index(['osm_id', 'name', 'other_tags', 'geometry', 'power'], dtype='object')
Index(['osm_id', 'name', 'other_tags', 'power', 'geometry'], dtype='object')


In [13]:
######################
# Merge polygon gdfs
######################

osm_power_polygons_concat = pd.concat([osm_power_polygons, osm_towers_and_poles], ignore_index=True)
osm_power_polygons_concat.reset_index(drop=True, inplace=True)

osm_power_polygons_concat.to_file('Y:/Forest Inventory/0700_NonCore_Funded/0726_TOW_Wales/04_Spatial Analysis/1_Reference_Data/11_OpenStreetMap/osm_power_structures.gpkg')